In [1]:
pip install git+https://github.com/mimoralea/gym-walk   #egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-req-build-2i72cfto
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-req-build-2i72cfto
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym_walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=2e41fd66080f5244687945a0a17746dfeb5c3b96ea9677114213b77191429125
  Stored in directory: /tmp/pip-ephem-wheel-cache-n9pz6ves/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym_walk


In [2]:
import warnings ; warnings.filterwarnings('ignore')

import gym
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123);

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [4]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print(" ".rjust(9), end="  ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [5]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [6]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [7]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [8]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [11]:
init_state

0

In [13]:
init_state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(init_state, reward, done, info))

state:0 - reward:0.0 - done:False - info:{'prob': 0.3333333333333333}


In [14]:
pi_frozenlake = lambda s: {
    0: RIGHT,
    1: DOWN,
    2: RIGHT,
    3: LEFT,
    4: DOWN,
    5: LEFT,
    6: RIGHT,
    7:LEFT,
    8: UP,
    9: DOWN,
    10:LEFT,
    11:DOWN,
    12:RIGHT,
    13:RIGHT,
    14:DOWN,
    15:LEFT #Stop
}[s]
print_policy(pi_frozenlake, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Policy:
| 00      > | 01      v | 02      > | 03      < |
| 04      v |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [15]:
print("Policy -> FrozenLake")
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake, goal_state=goal_state) * 100,
    mean_return(env, pi_frozenlake)))

Policy -> FrozenLake
Reaches goal 10.00%. Obtains an average undiscounted return of 0.1000.


In [16]:
pi_2 = lambda s: {
    0: RIGHT,
    1: RIGHT,
    2: RIGHT,
    3: DOWN,
    4: DOWN,
    5: RIGHT,
    6: DOWN,
    7: LEFT,
    8: RIGHT,
    9: DOWN,
    10: RIGHT,
    11: LEFT,
    12: RIGHT,
    13: RIGHT,
    14: DOWN,
    15: LEFT
}[s]
print("Name: Krithick Vivekananda ")
print("Register Number: 212223240075 ")
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Name: Krithick Vivekananda 
Register Number: 212223240075 
Policy:
| 00      > | 01      > | 02      > | 03      v |
| 04      v |           | 06      v |           |
| 08      > | 09      v | 10      > |           |
|           | 13      > | 14      v |           |


In [17]:
print("Policy -> 2")
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state) * 100,
    mean_return(env, pi_2)))

Policy -> 2
Reaches goal 5.00%. Obtains an average undiscounted return of 0.0500.


Sure! Here's the comparison in **single-line points**:

---

1. `Policy → FrozenLake` has a **10% success rate**, while `Policy → 2` has **5%**.
2. `Policy → FrozenLake` has a **0.1000 average return**, vs. **0.0500** for `Policy → 2`.
3. `Policy → FrozenLake` uses **varied directions**; `Policy → 2` is mostly **RIGHT/DOWN**.
4. `Policy → FrozenLake` is **more adaptable and avoids holes better**.
5. ✅ **`Policy → FrozenLake` performs better overall.**


In [18]:
import numpy as np
def policy_evaluation(pi,P,gamma=1.0,theta=1e-10):
  prev_V=np.zeros(len(P))
  while True:
    V=np.zeros(len(P))
    for s in range(len(P)):
      for prob, next_state, reward, done in P[s][pi(s)]:
        V[s]+=prob*(reward+gamma*prev_V[next_state]*(not done))
    if np.max(np.abs(prev_V-V)) < theta:
      break
    prev_V=V.copy()
  return V

In [19]:
V1 = policy_evaluation(pi_frozenlake, P,gamma=0.99)
print("Policy -> FrozenLake")
print_state_value_function(V1, P, n_cols=4, prec=5)

Policy -> FrozenLake
State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |            | 06 0.20562 |            |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |            |
|            | 13 0.62915 | 14 0.80739 |            |


In [20]:
V2 = policy_evaluation(pi_2, P,gamma=0.99)
print("Policy -> 2")
print_state_value_function(V1, P, n_cols=4, prec=5)

Policy -> 2
State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |            | 06 0.20562 |            |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |            |
|            | 13 0.62915 | 14 0.80739 |            |


Here is a **simple 5-point comparison** of the **state-value functions** for `pi_frozenlake` and `pi_2` based on your output:

---

1. **Overall values are higher** for `pi_frozenlake` than for `pi_2`.
2. `pi_frozenlake` has high values in key states like 6 (0.20562), 8 (0.30562), 14 (0.80739).
3. `pi_2` shows lower values in the same states: 6 (0.09215), 8 (0.1196), 14 (0.75408).
4. `pi_frozenlake` maintains better value propagation across the grid.
5. ✅ **`pi_frozenlake` has a better state-value function, indicating a stronger policy.**


In [22]:
V1>=V2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [23]:
if(np.sum(V1>=V2)>=11):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)>=11):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

The first policy is the better policy
